# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
i = 0
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
file_path_list

/home/workspace


['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # return the next row, 즉 head skip하기 

 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
#csv.QUOTE_ALL은 '' 안에 들어있는 모든걸 하나로 묶겠다는 의미
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # cassandra에 들어갈 column select
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION={
                    'class' : 'SimpleStrategy',
                    'replication_factor': 1
                    }""")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.execute("""USE udacity""")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Create table to query songs by sessionId and itemInSession. Set Primary Key to (sessionId and itemInSession)
session.execute("""CREATE TABLE IF NOT EXISTS udacity.songs_by_session(
                    sessionId int,
                    itemInSession int,
                    artist text,
                    length decimal,  
                    song text,
                    PRIMARY KEY (sessionId, itemInSession)
                    )
                """)

                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.songs_by_session (sessionId, itemInSession, artist, length, song)"""
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Verify the data by calculating the count of rows which should be 6820
rows = session.execute("SELECT count(*) FROM songs_by_session")
for row in rows:
    print(row)

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
rows = session.execute("SELECT artist, song, length FROM songs_by_session WHERE sessionId = 338 AND itemInSession = 4")
for row in rows:
    print(row.artist, ' ', row.song, ' ', row.length, ' ')


Row(count=6820)
Faithless   Music Matters (Mark Knight Dub)   495.3073  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Create table to query songs by user information. I set partition key to (userId, sessionId) and clustering key to itemInSession to get song sorted by itemInSession
session.execute("""CREATE TABLE IF NOT EXISTS udacity.songs_by_user(
                    userId int,
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song text,
                    firstName text,
                    lastName text,    
                    PRIMARY KEY ((userId, sessionId), itemInSession)
                    )
                """)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.songs_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"""
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## Verify the data by calculating the count of rows which should be 6820
rows = session.execute("SELECT count(*) FROM songs_by_user")
for row in rows:
    print(row)

    
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182    
rows = session.execute("SELECT artist, song, firstName, lastName FROM songs_by_user WHERE userId = 10 AND sessionId = 182")
for row in rows:
    print(row)

Row(count=6820)
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Create table to query every user name who listened to specific song. Set partition key to song and clustering key to userId to avoid duplication
session.execute("""CREATE TABLE IF NOT EXISTS udacity.users_by_song(
                    song text,
                    userId int, 
                    sessionId int,
                    itemInSession int,
                    firstName text,
                    lastName text,
                    PRIMARY KEY (song, userId)
                    )
                """)

                    

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO udacity.users_by_song (song, userId, sessionId, itemInSession, firstName, lastName)"""
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"

        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))

In [45]:
## Verify the data by calculating the count of rows which should be 6820
rows = session.execute("SELECT count(*) FROM users_by_song")
for row in rows:
    print(row)
    
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
rows = session.execute("SELECT firstName, lastName FROM users_by_song WHERE song = 'All Hands Against His Own'")
for row in rows:
    print(row)

Row(count=6820)
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [22]:
## TO-DO: Drop the table before closing out the sessions
session.execute("""DROP TABLE songs_by_session""")
session.execute("""DROP TABLE songs_by_user""")
session.execute("""DROP TABLE users_by_song""")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()